In [ ]:
!pip install kaggle
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

from google.colab import drive
drive.mount('./gdrive')

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 64 bytes
Mounted at ./gdrive


In [ ]:
!pip install -q pytorch-lightning & pip install -q -U albumentations & pip install -q iterative-stratification
!pip install -q timm & pip install -q einops & pip install -q pytorch-lightning wandb & pip install torch-ema
!git clone https://github.com/mlpc-ucsd/CoaT

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.8/225.8 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.4/291.4 kB 18.6 MB/s eta 0:00:00
Cloning into 'CoaT'...
remote: Enumerating objects: 1339, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 1339 (delta 10), reused 8 (delta 8), pack-reused 1325 (from 1)
Receiving objects: 100% (1339/1339), 7.10 MiB | 22.10 MiB/s, done.
Resolving deltas: 100% (464/464), done.


In [ ]:
!pip install -q pydicom==2.4.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 48.7 MB/s eta 0:00:00


In [ ]:
%%time
!kaggle competitions download -c rsna-2024-lumbar-spine-degenerative-classification
!unzip -q rsna-2024-lumbar-spine-degenerative-classification.zip

100% 28.2G/28.2G [21:57<00:00, 23.5MB/s]
100% 28.2G/28.2G [21:57<00:00, 23.0MB/s]
CPU times: user 14.6 s, sys: 1.93 s, total: 16.6 s
Wall time: 28min 40s


In [ ]:
!kaggle datasets download -d wadakoki/rsna-newmeta
!unzip -q /content/rsna-newmeta.zip

Dataset URL: https://www.kaggle.com/datasets/wadakoki/rsna-newmeta
License(s): unknown
100% 3.76M/3.76M [00:01<00:00, 4.21MB/s]
100% 3.76M/3.76M [00:01<00:00, 3.09MB/s]


In [ ]:
import wandb
from pytorch_lightning.loggers import WandbLogger
import os
import yaml
import sys
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from glob import glob
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.optim import AdamW, Adam
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR, ReduceLROnPlateau
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar
import torchvision.transforms as T
import albumentations as A
import pandas.api.types
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold, StratifiedGroupKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
#from torchvision.models.maxvit import MaxVit
from timm.models.maxxvit import MaxxVit
import timm
import scipy
import albumentations as A
from torchvision.transforms import v2
from torchvision import models
from einops import repeat
from einops.layers.torch import Rearrange
from tqdm.auto import tqdm
sys.path.append('/content/CoaT')
from CoaT.src.models.coat import coat_lite_medium
from joblib import Parallel, delayed
from torch.utils.data import default_collate
from torch_ema import ExponentialMovingAverage
import pydicom as dcm
import transformers

/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.10/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/content/CoaT/src/models/coat.py:589: UserWarning: Overwriting coat_tiny in registry with CoaT.src.models.coat.coat_tiny. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def coat_tiny(**kwargs):
/content/CoaT/src/models/coat.py:595: UserWarning: Overwriting coat_mini in registry with CoaT.src.models.coat.coat_mini. This is because the name being registered conflicts with an existing nam

## training

In [ ]:
SEED = 126 # friend's birthday
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True # Fix the network according to random seed
    print('Finish seeding with seed {}'.format(seed))

seed_everything(SEED)
print('Training on device {}'.format(device))

Finish seeding with seed 126
Training on device cuda


In [ ]:
%%writefile config.yaml

data_path: "/content/"
output_dir: "/content/gdrive/MyDrive/RSNA_SPINE/models/"

seed: 1101

train_bs: 4
valid_bs: 4
workers: 1

progress_bar_refresh_rate: 1

pseudo_train: 0

save_topk: 1
fold: 5

task:
    kind: 'detect'
    #kind: 'classify'
    #kind: 'depth'
    #condition: 'nfn'
    condition: 'scs'
    #condition: 'ss'

in_chans: 3

image_size: 384
early_stop:
    monitor: "val_loss"
    mode: "min"
    patience: 999
    verbose: 1

trainer:
    max_epochs: 30
    min_epochs: 30
    enable_progress_bar: True
    precision: "16-mixed"
    devices: 1

model:
    name: "eff"
    loss_smooth: 0.0
    optimizer_params:
        #lr: 0.001
        lr: 0.0001
        weight_decay: 0.0001
    scheduler:
        #name: "CosineAnnealingLR"
        name: "cosine_with_warmup"
        params:
            CosineAnnealingLR:
                T_max: 20
                eta_min: 1.0e-5
                last_epoch: -1
            cosine_with_warmup:
                num_training_steps: 30
                num_warmup_steps: 1
                num_cycles: 0.5
                last_epoch: -1


Overwriting config.yaml


In [ ]:
with open("config.yaml", "r") as file_obj:
    config = yaml.safe_load(file_obj)

In [ ]:
class CoorDetectDataset(Dataset):
    def __init__(self, coor, meta, condition, usage='train'):
        if condition == 'scs':
            coor = coor.loc[coor.condition=='Spinal Canal Stenosis']
        elif condition == 'ss':
            coor = coor.loc[(coor.condition=='Left Subarticular Stenosis') | (coor.condition=='Right Subarticular Stenosis')]
        elif condition == 'nfn':
            coor = coor.loc[(coor.condition=='Right Neural Foraminal Narrowing') | (coor.condition=='Left Neural Foraminal Narrowing')]
        g_coor = coor.groupby('study_id').count()
        if condition == 'scs':
            self.id = g_coor.loc[g_coor.series_id==5].reset_index().study_id.unique()
        else:
            self.id = g_coor.loc[g_coor.series_id==10].reset_index().study_id.unique()
        self.coor = coor
        self.meta = meta
        self.condition = condition
        self.usage = usage
        self.id = list(set(self.id) - set([3637444890]))
        #self.id = [2773343225]
        #self.id = [1782095928]

        self.resize = v2.Resize((384, 384))

    def __getitem__(self, index):
        study_id = self.id[index]
        #print(study_id)
        #try:
        if self.condition == 'scs':
            volume, label = self.for_scs(study_id)
        elif self.condition == 'nfn':
            volume, label = self.for_nfn(study_id)
        if self.condition == 'ss':
            volume, label = self.for_ss(study_id)
        return volume, label, torch.tensor(study_id)

    def for_scs(self, study_id):
        meta = self.meta.loc[(self.meta.study_id==study_id) & (self.meta.series_description=='Sagittal T2/STIR')]
        meta = meta.sort_values('ipp_x', ascending=True).reset_index(drop=True)
        #img = [self.normalize(self.load_dicom(f'/content/train_images/{row.study_id}/{row.series_id}/{row.instance_number}.dcm')) for _, row in meta.iterrows()]
        coor = self.coor.loc[(self.coor.study_id==study_id) & (self.coor.condition=='Spinal Canal Stenosis')]
        coor_dict = {}
        meta_list = []
        for _, row in coor.iterrows():
            series_id, instance_number = row.series_id, row.instance_number
            meta_list.append(meta.loc[(meta.series_id==series_id) & (meta.instance_number==instance_number)])
        sub_meta = pd.concat(meta_list)
        idx = meta.loc[meta.ipp_x == sub_meta.ipp_x.median()].index[0]
        #print(old_idx)
        img_row = meta.iloc[idx]
        before_img_row = meta.iloc[idx-1]
        after_img_row = meta.iloc[idx+1]
        img = self.normalize(self.load_dicom(f'/content/train_images/{img_row.study_id}/{img_row.series_id}/{img_row.instance_number}.dcm'))
        bimg = self.normalize(self.load_dicom(f'/content/train_images/{before_img_row.study_id}/{before_img_row.series_id}/{before_img_row.instance_number}.dcm'))
        aimg = self.normalize(self.load_dicom(f'/content/train_images/{after_img_row.study_id}/{after_img_row.series_id}/{after_img_row.instance_number}.dcm'))
        height, width = img.shape
        for _, row in coor.iterrows():
            series_id, instance_number = row.series_id, row.instance_number
            #x = round((row.x/width)*384)
            #y = round((row.y/height)*384)
            x = row.x/width
            y = row.y/height
            coor_dict[row.level] = torch.tensor([x, y]).to(torch.float32)
        img = self.resize(torch.tensor(img[None, ...]))
        bimg = self.resize(torch.tensor(bimg[None, ...]))
        aimg = self.resize(torch.tensor(aimg[None, ...]))
        img = torch.cat([bimg, img, aimg]).to(torch.float32)
        return img, coor_dict

    def for_nfn(self, study_id):
        meta = self.meta.loc[(self.meta.study_id==study_id) & (self.meta.series_description=='Sagittal T1')]
        meta = meta.sort_values('ipp_x', ascending=True).reset_index(drop=True)
        #img = [self.normalize(self.load_dicom(f'/content/train_images/{row.study_id}/{row.series_id}/{row.instance_number}.dcm')) for _, row in meta.iterrows()]
        coor = self.coor.loc[(self.coor.study_id==study_id)]
        coor_dict = {}
        right_meta_list = []
        left_meta_list = []
        for _, row in coor.iterrows():
            series_id, instance_number = row.series_id, row.instance_number
            if row.condition == 'Right Neural Foraminal Narrowing':
                right_meta_list.append(meta.loc[(meta.series_id==series_id) & (meta.instance_number==instance_number)])
            else:
                left_meta_list.append(meta.loc[(meta.series_id==series_id) & (meta.instance_number==instance_number)])

        right_sub_meta = pd.concat(right_meta_list)
        left_sub_meta = pd.concat(left_meta_list)
        ridx = meta.loc[meta.ipp_x == right_sub_meta.ipp_x.median()].index[0]
        lidx = meta.loc[meta.ipp_x == left_sub_meta.ipp_x.median()].index[0]
        #display(right_sub_meta)
        #display(coor)
        if self.usage == 'train':
            ridx_shift = random.randint(-2, 2)
            lidx_shift = random.randint(-2, 2)
        else:
            ridx_shift = 0
            lidx_shift = 0
        right_img_row = meta.iloc[min(max(ridx+ridx_shift, 0), len(meta)-1)]
        #display(right_img_row)
        right_before_img_row = meta.iloc[min(max(ridx-1+ridx_shift, 0), len(meta)-1)]
        rightafter_img_row = meta.iloc[min(max(ridx+1+ridx_shift, 0), len(meta)-1)]
        left_img_row = meta.iloc[min(max(lidx+lidx_shift, 0), len(meta)-1)]
        left_before_img_row = meta.iloc[min(max(lidx-1+lidx_shift, 0), len(meta)-1)]
        leftafter_img_row = meta.iloc[min(max(lidx+1+lidx_shift, 0), len(meta)-1)]
        rimg = self.normalize(self.load_dicom(f'/content/train_images/{right_img_row.study_id}/{right_img_row.series_id}/{right_img_row.instance_number}.dcm'))
        rbimg = self.normalize(self.load_dicom(f'/content/train_images/{right_before_img_row.study_id}/{right_before_img_row.series_id}/{right_before_img_row.instance_number}.dcm'))
        raimg = self.normalize(self.load_dicom(f'/content/train_images/{rightafter_img_row.study_id}/{rightafter_img_row.series_id}/{rightafter_img_row.instance_number}.dcm'))
        limg = self.normalize(self.load_dicom(f'/content/train_images/{left_img_row.study_id}/{left_img_row.series_id}/{left_img_row.instance_number}.dcm'))
        lbimg = self.normalize(self.load_dicom(f'/content/train_images/{left_before_img_row.study_id}/{left_before_img_row.series_id}/{left_before_img_row.instance_number}.dcm'))
        laimg = self.normalize(self.load_dicom(f'/content/train_images/{leftafter_img_row.study_id}/{leftafter_img_row.series_id}/{leftafter_img_row.instance_number}.dcm'))
        rheight, rwidth = rimg.shape
        lheight, lwidth = limg.shape
        for _, row in coor.iterrows():
            series_id, instance_number = row.series_id, row.instance_number
            if row.condition == 'Right Neural Foraminal Narrowing':
                x = row.x/rwidth
                y = row.y/rheight
                coor_dict['right_' + row.level] = torch.tensor([x, y]).to(torch.float32)
            else:
                x = row.x/lwidth
                y = row.y/lheight
                coor_dict['left_' + row.level] = torch.tensor([x, y]).to(torch.float32)
        rimg = torch.cat([self.resize(torch.tensor(i)[None, ...]).to(torch.float32) for i in [rbimg, rimg, raimg]])
        limg = torch.cat([self.resize(torch.tensor(i)[None, ...]).to(torch.float32) for i in [lbimg, limg, laimg]])
        img = torch.stack([limg, rimg]).to(torch.float32).contiguous()
        return img, coor_dict

    def normalize(self, x):
        lower, upper = np.percentile(x, (1, 99))
        x = np.clip(x, lower, upper)
        x = x - np.min(x)
        x = x / np.max(x)
        return x

    def __len__(self):
        return len(self.id)

    def load_dicom(self, path):
        dicom = dcm.read_file(path)
        data = dicom.pixel_array
        return data

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class SCSDetectLoss(nn.Module):
    def __init__(self):
        super(SCSDetectLoss, self).__init__()

    def forward(self, outputs, targets):
        loss = 0
        for level in ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']:
            _loss = nn.functional.l1_loss(outputs[level], targets[level])
            loss += _loss
        return loss/5
class _SCSDetectLoss(nn.Module):
    def __init__(self):
        super(SCSDetectLoss, self).__init__()
        self.map = {'x': 0, 'y': 1}
    def forward(self, outputs, targets):
        loss = 0
        for level in ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']:
            for c in ['x', 'y']:
                _loss = nn.functional.cross_entropy(outputs[level][c], targets[level][:, self.map[c]].to(torch.long))
                loss += _loss
        return loss/10

class NFNSSDetectLoss(nn.Module):
    def __init__(self):
        super(NFNSSDetectLoss, self).__init__()

    def forward(self, outputs, targets):
        loss = 0
        for level in ['left_L1/L2', 'left_L2/L3', 'left_L3/L4', 'left_L4/L5', 'left_L5/S1',
                  'right_L1/L2', 'right_L2/L3', 'right_L3/L4', 'right_L4/L5', 'right_L5/S1']:
            _loss = nn.functional.l1_loss(outputs[level], targets[level])
            loss += _loss
        return loss/10

class SCSDepthDetectLoss(nn.Module):
    def __init__(self):
        super(SCSDepthDetectLoss, self).__init__()

    def forward(self, outputs, targets):
        loss = 0
        for level in ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']:
            #print(outputs[level].shape, targets[level].shape)
            _loss = nn.functional.cross_entropy(outputs[level], targets[level].reshape(-1))
            loss += _loss
        return loss/5

class NFNSSDepthDetectLoss(nn.Module):
    def __init__(self):
        super(NFNSSDepthDetectLoss, self).__init__()

    def forward(self, outputs, targets):
        loss = 0
        for level in ['left_L1/L2', 'left_L2/L3', 'left_L3/L4', 'left_L4/L5', 'left_L5/S1',
                  'right_L1/L2', 'right_L2/L3', 'right_L3/L4', 'right_L4/L5', 'right_L5/S1']:
            _loss = nn.functional.cross_entropy(outputs[level], targets[level].reshape(-1))
            loss += _loss
        return loss/10

class SCSNFNSSLoss(nn.Module):
    def __init__(self, is_train=False):
        super(SCSNFNSSLoss, self).__init__()
        self.loss = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 4.0]).to(device))
        self.aux_loss_ax = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 4.0]).to(device))
        self.aux_loss_sagt2 = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 4.0]).to(device))
        self.aux_loss_sagt1 = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 4.0]).to(device))
        self.is_train = is_train
    def forward(self, outputs, targets, ax=None, sagt2=None, sagt1=None):
        targets = targets.reshape(-1)
        #print(outputs, targets, outputs.shape, targets.shape)
        ax_loss = 0
        sagt2_loss = 0
        sagt1_loss = 0
        num_loss = 1
        loss = self.loss(outputs, targets)
        if ax is not None:
            ax_loss = self.aux_loss_ax(ax, targets)
            if self.is_train:
                loss += ax_loss*0.5
                num_loss += 1
        if sagt2 is not None:
            sagt2_loss = self.aux_loss_sagt2(sagt2, targets)
            if self.is_train:
                loss += sagt2_loss*0.5
                num_loss += 1
        if sagt1 is not None:
            sagt1_loss = self.aux_loss_sagt1(sagt1, targets)
            if self.is_train:
                loss += sagt1_loss*0.5
                num_loss += 1
        #print(loss)
        loss = loss/num_loss
        return loss, ax_loss, sagt2_loss, sagt1_loss

In [ ]:
class ConvNextSCSDetect(nn.Module):
    def __init__(self):
        super().__init__()
        #self.size = 384
        self.encoder = timm.create_model('convnext_base.fb_in22k_ft_in1k_384', in_chans=3, pretrained=True, num_classes=0, drop_rate=0.)
        self.in_features = self.encoder.num_features
        self.flatten = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)),
                                    nn.Flatten(1),
                                    #nn.LayerNorm(self.in_features)
                                    )

        self.l1 = nn.Linear(self.in_features, 2)
        self.l2 = nn.Linear(self.in_features, 2)
        self.l3 = nn.Linear(self.in_features, 2)
        self.l4 = nn.Linear(self.in_features, 2)
        self.l5 = nn.Linear(self.in_features, 2)

    def forward(self, x, label=None):
        #for loc, img in x.items():
            #print(img.shape)
        #    img = self.encoder.forward_features(img)
        #    img = self.flatten(img)
        #    x[loc] = img
        x = self.encoder.forward_features(x)
        x = self.flatten(x)
        l1 = self.l1(x)
        l2 = self.l2(x)
        l3 = self.l3(x)
        l4 = self.l4(x)
        l5 = self.l5(x)
        return {'L1/L2': l1.sigmoid(), 'L2/L3': l2.sigmoid(), 'L3/L4': l3.sigmoid(), 'L4/L5': l4.sigmoid(), 'L5/S1': l5.sigmoid()}


class ConvNextNFNDetect(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = timm.create_model('convnext_base.fb_in22k_ft_in1k_384', in_chans=3, pretrained=True, num_classes=0)
        self.in_features = self.encoder.num_features
        self.flatten = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)),
                                    nn.Flatten(1),
                                    #nn.LayerNorm(self.in_features)
                                    )
        self.ll1 = nn.Linear(self.in_features, 2)
        self.ll2 = nn.Linear(self.in_features, 2)
        self.ll3 = nn.Linear(self.in_features, 2)
        self.ll4 = nn.Linear(self.in_features, 2)
        self.ll5 = nn.Linear(self.in_features, 2)
        self.rl1 = nn.Linear(self.in_features, 2)
        self.rl2 = nn.Linear(self.in_features, 2)
        self.rl3 = nn.Linear(self.in_features, 2)
        self.rl4 = nn.Linear(self.in_features, 2)
        self.rl5 = nn.Linear(self.in_features, 2)
    def forward(self, x, label=None):
        shape = x.shape
        x = x.reshape(shape[0]*shape[1], 3, shape[-2], shape[-1])
        x = self.encoder.forward_features(x)
        x = self.flatten(x)
        x = x.reshape(shape[0], shape[1], -1)
        x_left = x[:, 0, :]
        x_right = x[:, 1, :]
        ll1 = self.ll1(x_left)
        ll2 = self.ll2(x_left)
        ll3 = self.ll3(x_left)
        ll4 = self.ll4(x_left)
        ll5 = self.ll5(x_left)
        rl1 = self.rl1(x_right)
        rl2 = self.rl2(x_right)
        rl3 = self.rl3(x_right)
        rl4 = self.rl4(x_right)
        rl5 = self.rl5(x_right)
        return {'left_L1/L2': ll1.sigmoid(),'left_L2/L3': ll2.sigmoid(),'left_L3/L4': ll3.sigmoid(), 'left_L4/L5': ll4.sigmoid(), 'left_L5/S1': ll5.sigmoid(),
                'right_L1/L2': rl1.sigmoid(), 'right_L2/L3': rl2.sigmoid(), 'right_L3/L4': rl3.sigmoid(), 'right_L4/L5': rl4.sigmoid(), 'right_L5/S1': rl5.sigmoid()}

In [ ]:
from traitlets.config.loader import ConfigError
VAL_SCORE = []

class DetectModule(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config
        if config['task']['condition'] == 'scs':
            self.model = ConvNextSCSDetect()
            self.loss_module = SCSDetectLoss()
        elif config['task']['condition'] == 'nfn':
            self.model = ConvNextNFNDetect()
            self.loss_module = NFNSSDetectLoss()
        elif  config['task']['condition'] == 'ss':
            self.model = SSDetect()
            self.loss_module = NFNSSDetectLoss()
        self.val_step_outputs = []
        self.val_step_labels = []
        #self.ema = ExponentialMovingAverage(self.model.parameters(), decay=0.995)
        #self.ema.to(device)

        #self.model = torch.optim.swa_utils.AveragedModel(self.model,
        #                                                 multi_avg_fn=torch.optim.swa_utils.get_ema_multi_avg_fn(0.999))

    def forward(self, batch):
        preds = self.model(batch)
        return preds

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), **self.config['model']["optimizer_params"])
        if self.config['model']["scheduler"]["name"] == "CosineAnnealingLR":
            scheduler = CosineAnnealingLR(
                optimizer,
                **self.config['model']["scheduler"]["params"]["CosineAnnealingLR"],
            )
            lr_scheduler_dict = {"scheduler": scheduler, "interval": "step"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler_dict}
        elif self.config['model']["scheduler"]["name"] == "ReduceLROnPlateau":
            scheduler = ReduceLROnPlateau(
                optimizer,
                **self.config['model']["scheduler"]["params"]["ReduceLROnPlateau"],
            )
            lr_scheduler = {"scheduler": scheduler, "monitor": "val_loss"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}
        elif self.config['model']['scheduler']['name'] == 'ChrisLR':
            scheduler = ChrisLR(
                optimizer,
                **self.config['model']["scheduler"]["params"]["ChrisLR"],
            )
            lr_scheduler = {"scheduler": scheduler, "monitor": "val_loss"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}
        elif self.config['model']['scheduler']['name'] == 'cosine_with_warmup':
            print('cosine with warmup')
            print(self.config['model']['scheduler']['params']['cosine_with_warmup'])
            scheduler = transformers.get_cosine_schedule_with_warmup(
                optimizer,
                **self.config['model']['scheduler']['params']['cosine_with_warmup'],
            )
            lr_scheduler_dict = {"scheduler": scheduler, "interval": "step"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler_dict}
        else:
            return {"optimizer": optimizer}

    def training_step(self, batch, batch_idx):
        imgs, label, _ = batch
        preds = self.model(imgs)
        loss = self.loss_module(preds, label)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, batch_size=config['train_bs'])
        for param_group in self.trainer.optimizers[0].param_groups:
            lr = param_group["lr"]
        self.log("lr", lr, on_step=True, on_epoch=False, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        """Add TTA"""
        volume, label, _ = batch
        preds = self.model.forward(volume)
        loss = self.loss_module(preds, label)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        #self.val_step_outputs.append(preds)
        #self.val_step_labels.append(labels)
        return loss

    def on_validation_epoch_end(self):
        if self.trainer.global_rank == 0:
            print(f"\nEpoch: {self.current_epoch}", flush=True)
        return 0

    def optimizer_step(self, *args, **kwargs):
        super().optimizer_step(*args, **kwargs)
        #self.ema.update()

In [ ]:
train = pd.read_csv('/content/train.csv')
train_dummy = pd.read_csv('/content/train.csv')
train_series = pd.read_csv('/content/train_series_descriptions.csv')
train_coor = pd.read_csv('/content/train_label_coordinates.csv')
train_meta = pd.read_parquet('/content/meta.parquet')

In [ ]:
train_dummy = train_dummy.fillna('Normal/Mild')

In [ ]:
label = train.columns[1:]
kfold = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for i, (train_idx, valid_idx) in enumerate(kfold.split(train_dummy, train_dummy[label])):
    train_dummy.loc[valid_idx, 'fold'] = i
train_series = train_series.merge(train_dummy[['study_id', 'fold']], on='study_id')
train_coor = train_coor.merge(train_dummy[['study_id', 'fold']], on='study_id')
train = train.merge(train_dummy[['study_id', 'fold']], on='study_id')
train_meta = train_meta.merge(train_dummy[['study_id', 'fold']], on='study_id')

In [ ]:
T_MAX = config["model"]["scheduler"]["params"]["CosineAnnealingLR"]["T_max"]
num_training_steps = config["model"]["scheduler"]["params"]["cosine_with_warmup"]["num_training_steps"]
num_warmup_steps = config["model"]["scheduler"]["params"]["cosine_with_warmup"]["num_warmup_steps"]

In [ ]:
%%time
import warnings
warnings.filterwarnings("ignore")
##############COORDINATE DETECT#########################
print(config['task']['condition'])
print(config['task']['kind'])
print(T_MAX)
#for i in [5]:
for i in [0, 1, 2, 3, 4]:
    train_df = train.loc[train.fold != i]
    valid_df = train.loc[train.fold == i]
    train_df = train_df.drop('fold', axis=1)
    valid_df = valid_df.drop('fold', axis=1)
    train_series_df = train_series.loc[train_series.fold != i]
    valid_series_df = train_series.loc[train_series.fold == i]
    train_series_df = train_series_df.drop('fold', axis=1)
    valid_series_df = valid_series_df.drop('fold', axis=1)
    train_coor_df = train_coor.loc[train_coor.fold != i]
    valid_coor_df = train_coor.loc[train_coor.fold == i]
    train_coor_df = train_coor_df.drop('fold', axis=1)
    valid_coor_df = valid_coor_df.drop('fold', axis=1)
    train_meta_df = train_meta.loc[train_meta.fold != i]
    valid_meta_df = train_meta.loc[train_meta.fold == i]
    train_meta_df = train_meta_df.drop('fold', axis=1)
    valid_meta_df = valid_meta_df.drop('fold', axis=1)
    if config['task']['condition'] != 'ss':
        dataset_train = CoorDetectDataset(train_coor_df, train_meta_df, config['task']['condition'], 'train')
        dataset_validation = CoorDetectDataset(valid_coor_df, valid_meta_df, config['task']['condition'], 'valid')

    else:
        dataset_train = SSDetectDataset(train_coor_df, train_meta_df, config['task']['condition'], 'train')
        dataset_validation = SSDetectDataset(valid_coor_df, valid_meta_df, config['task']['condition'], 'valid')
        config['train_bs'] = 2
    data_loader_train = DataLoader(
        dataset_train,
        batch_size=config["train_bs"],
        shuffle=True,
        num_workers=4,
        pin_memory=True,
    )
    data_loader_validation = DataLoader(
        dataset_validation,
        batch_size=config["valid_bs"],
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )

    checkpoint_callback = ModelCheckpoint(
        save_weights_only=True,
        monitor="val_loss",
        dirpath=config["output_dir"],
        mode='min',
        filename=f"{config['task']['condition']}_{config['task']['kind']}_pre_{i}",
        save_top_k=config["save_topk"],
        verbose=1,
    )

    progress_bar_callback = TQDMProgressBar(
        refresh_rate=config["progress_bar_refresh_rate"]
    )

    early_stop_callback = EarlyStopping(**config["early_stop"])
    _c, _k = config['task']['condition'], config['task']['kind']
    wandb_logger = WandbLogger(project=f'rsna_spine_{_c}_{_k}', # group runs in "MNIST" project
                            log_model=False) # log all new checkpoints during training

    trainer = pl.Trainer(
        logger=wandb_logger,
        callbacks=[checkpoint_callback, early_stop_callback, progress_bar_callback],
        **config["trainer"],
    )

    config["model"]["scheduler"]["params"]["CosineAnnealingLR"]["T_max"] = T_MAX*len(data_loader_train)/config["trainer"]["devices"]
    config["model"]["scheduler"]["params"]["cosine_with_warmup"]["num_training_steps"] = int(num_training_steps*len(data_loader_train)/config["trainer"]["devices"])
    config["model"]["scheduler"]["params"]["cosine_with_warmup"]["num_warmup_steps"] = int(num_warmup_steps*len(data_loader_train)/config["trainer"]["devices"])
    if config['task']['condition'] == 'scs':
        model = DetectModule.load_from_checkpoint('/content/gdrive/MyDrive/RSNA_SPINE/pretrained_models/scs_detect_pretrained_convnext-base-1e-4.ckpt', strict=False, config=config)
    else:
        model = DetectModule(config=config)
    trainer.fit(model, data_loader_train, data_loader_validation)
    wandb.finish()

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


scs
detect
20


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: syujinko. Use `wandb login --relogin` to force relogin


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type              | Params | Mode 
----------------------------------------------------------
0 | model       | ConvNextSCSDetect | 87.6 M | train
1 | loss_module | SCSDetectLoss     | 0      | train
----------------------------------------------------------
87.6 M    Trainable params
0         Non-trainable params
87.6 M    Total params
350.307   Total estimated model params size (MB)
475       Modules in train mode
0         Modules in eval mode


cosine with warmup
{'num_training_steps': 11400, 'num_warmup_steps': 380, 'num_cycles': 0.5, 'last_epoch': -1}


...

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]


Epoch: 0


INFO:pytorch_lightning.callbacks.early_stopping:Metric val_loss improved. New best score: 0.012
INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 380: 'val_loss' reached 0.01234 (best 0.01234), saving model to '/content/gdrive/MyDrive/RSNA_SPINE/models/scs_detect_pre_1-v2.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined